Data source: [https://www.kaggle.com/datasets/kanchana1990/colorado-healthcare-decoding-no-show-pattern](https:\www.kaggle.com\datasets\kanchana1990\colorado-healthcare-decoding-no-show-pattern)s

In [19]:
SELECT TOP 10
    *
FROM
    test.dbo.healthcare_dataset

(10 rows affected)

Total execution time: 00:00:00.006

Patient_ID,Age,Gender,Insurance_Type,Chronic_Disease,Mental_Health_Status,Employment_Status,Education_Level,Transportation_Access,Distance_from_Facility,Area_Type,Booking_Date,Appointment_Date,Appointment_Outcome
75763,93,Male,Medicare,0,1,Student,Advanced Degree,1,37.78503801701182,Suburban,2023-11-04,2024-08-15,Show
84945,52,Male,Uninsured,0,0,Employed,College,1,NULL,Rural,2023-10-28,2024-07-11,No-Show
88559,8,Male,Medicare,1,NULL,NULL,Advanced Degree,1,27.277653823064092,Rural,2023-02-12,2024-10-26,Show
72217,5,Other,Medicare,0,1,Student,College,1,44.618708404834905,NULL,2023-01-30,2024-01-31,Show
44071,76,Male,Uninsured,1,0,Employed,Advanced Degree,0,12.568496687483455,Urban,2023-11-27,2024-03-03,NULL
76828,86,NULL,NULL,0,0,Retired,High School,1,27.24322102908982,NULL,2023-02-03,2023-12-22,NULL
34403,15,Female,Medicaid,NULL,0,Unemployed,High School,1,34.033755495646595,Suburban,2023-08-13,2024-07-23,Show
80253,68,NULL,Uninsured,0,1,Employed,None,0,46.61934145581211,Rural,2024-08-01,2024-10-22,No-Show
65628,43,Female,Private,1,1,Student,None,1,NULL,NULL,2022-11-24,2024-09-22,Show
99220,7,Male,Uninsured,NULL,0,Student,High School,1,46.55755754570776,Suburban,2023-03-15,2024-09-30,Show


### Basic Descriptive Analysis

**1\. What is the overall no-show rate?**

In [20]:
 WITH No_Show_Table AS(
    SELECT
        COUNT(Appointment_Outcome)
        AS No_Show_Appointments
    FROM
        test.dbo.healthcare_dataset
    WHERE
        Appointment_Outcome = 'No-Show'
 ),
Total_Apponitments_Table AS (
    SELECT
        COUNT(Appointment_Outcome)
        AS Total_Appointments
    FROM
        test.dbo.healthcare_dataset

)
SELECT
    a.No_Show_Appointments,
    b.Total_Appointments,
    CAST(a.No_Show_Appointments AS decimal(10,2)) / CAST(b.Total_Appointments AS decimal(10,2)) * 100
AS 'No-show rate'
FROM
    No_Show_Table a, Total_Apponitments_Table b

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.020

No_Show_Appointments,Total_Appointments,No-show rate
22467,45042,49.8801118955600


2. **How does no-show rate vary by day of the week?**

In [21]:
WITH No_Show_ByDay AS( SELECT
    CASE
        WHEN DATEPART(weekday, Appointment_Date) = 1 THEN 'Sunday'
        WHEN DATEPART(weekday, Appointment_Date) = 2 THEN 'Monday'
        WHEN DATEPART(weekday, Appointment_Date) = 3 THEN 'Tuesday'
        WHEN DATEPART(weekday, Appointment_Date) = 4 THEN 'Wednesday'
        WHEN DATEPART(weekday, Appointment_Date) = 5 THEN 'Thursday'
        WHEN DATEPART(weekday, Appointment_Date) = 6 THEN 'Friday'
        WHEN DATEPART(weekday, Appointment_Date) = 7 THEN 'Saturday'
    END AS DayOfWeekName,
    COUNT(*) AS NoShowCount

FROM
    test.dbo.healthcare_dataset
WHERE
    Appointment_Outcome = 'No-Show'
GROUP BY
    DATEPART(weekday, Appointment_Date)
),
No_Shows AS(

    SELECT
        COUNT(Appointment_Outcome) AS Total_Noshows
    FROM
        test.dbo.healthcare_dataset
    WHERE
        Appointment_Outcome = 'No-Show'

)
SELECT
    a.DayOfWeekName, CAST(CAST(a.NoShowCount AS decimal(10,2)) / b.Total_Noshows AS decimal(4,3)) * 100 AS 'No-Show rate by Date in %'
FROM
    No_Show_ByDay a, No_Shows b
ORDER BY 
    [No-Show rate by Date in %]
    DESC

(7 rows affected)

Total execution time: 00:00:00.050

DayOfWeekName,No-Show rate by Date in %
Wednesday,14.500
Thursday,14.400
Sunday,14.400
Friday,14.300
Saturday,14.200
Tuesday,14.200
Monday,13.900


### Demographic Analysis

**How do no-show rates differ across various age groups (****<span style="color: var(--vscode-foreground);">0-18, 19-35, 36-55, 56+)</span><span style="color: var(--vscode-foreground);">?</span>**

In [22]:
WITH AgeGroupTable AS (
    SELECT
        CASE
            WHEN Age <= 18 THEN '0-18'
            WHEN Age >= 19 AND Age < 35 THEN '19-35'
            WHEN Age >= 36 AND Age < 55 THEN '36-55'
            WHEN Age >= 56 THEN '56+'
        END AS AgeGroup,
        COUNT(*) AS NoShowCount
    FROM
        test.dbo.healthcare_dataset
    WHERE
        Appointment_Outcome = 'No-Show'
    GROUP BY
        CASE
            WHEN Age <= 18 THEN '0-18'
            WHEN Age >= 19 AND Age < 35 THEN '19-35'
            WHEN Age >= 36 AND Age < 55 THEN '36-55'
            WHEN Age >= 56 THEN '56+'
        END
),
No_Shows AS (
    SELECT
        COUNT(Appointment_Outcome) AS Total_Noshows
    FROM
        test.dbo.healthcare_dataset
    WHERE
        Appointment_Outcome = 'No-Show'
)
SELECT
    a.AgeGroup,
    CAST(a.NoShowCount AS decimal(10,2)) / b.Total_Noshows * 100 AS '% of No-Shows'
FROM
    AgeGroupTable a, No_Shows b
ORDER BY
    '% of No-Shows'
    DESC

(5 rows affected)

Total execution time: 00:00:00.060

AgeGroup,% of No-Shows
56+,39.5557929407500
0-18,17.0205189833900
36-55,16.7445586860700
19-35,14.7149152089700
NULL,11.9642141807900


**Is there a difference in no-show rates between genders?**

In [23]:
WITH GenderGroup AS (SELECT
    Gender,
    COUNT (*) AS 'No_Show'
FROM
    test.dbo.healthcare_dataset
WHERE
    Appointment_Outcome = 'No-Show'
GROUP by
    Gender),
GenderTotal AS(SELECT
    COUNT(*) AS No_ShowTotal
    FROM
        test.dbo.healthcare_dataset
    WHERE
        Appointment_Outcome = 'No-Show'
)
SELECT
    a.Gender,
    CAST(a.No_Show AS decimal(10,2)) / b.No_ShowTotal * 100 AS 'No-Show rate in %'
FROM
    GenderGroup a, GenderTotal b

(4 rows affected)

Total execution time: 00:00:00.042

Gender,No-Show rate in %
NULL,9.8767080607100
Female,30.3778875684300
Male,29.9105354519900
Other,29.8348689188500


### Time-based Patterns

**How do no-show rates vary between appointments made on the same day vs. days in advance?**

In [24]:
SELECT
    Booking_date,
    Appointment_date
FROM
    test.dbo.healthcare_dataset
WHERE
    Appointment_Date = Booking_Date
-- Result is blank, there are no same date appointments 

(0 rows affected)

Total execution time: 00:00:00.007

Booking_date,Appointment_date


### Correlation and Causal Analysis

**Are longer wait times between scheduling and appointment associated with higher no-show rates?**

In [15]:
SELECT
    DATEDIFF(day,Booking_Date,Appointment_Date) AS 'Waiting time in days',
    COUNT(DATEDIFF(day,Booking_Date,Appointment_Date)) AS 'Count',
    Appointment_Outcome
FROM
    test.dbo.healthcare_dataset
WHERE
    Appointment_Outcome = 'No-Show'
GROUP BY
    DATEDIFF(day,Booking_Date,Appointment_Date),
    Appointment_Outcome
ORDER BY
    COUNT(DATEDIFF(day,Booking_Date,Appointment_Date))
    DESC

    

(718 rows affected)

Total execution time: 00:00:00.048

Waiting time in days,Count,Appointment_Outcome
20,61,No-Show
106,60,No-Show
272,60,No-Show
273,59,No-Show
79,58,No-Show
123,58,No-Show
304,57,No-Show
29,56,No-Show
31,56,No-Show
230,56,No-Show
